In [1]:
"""
First lets get the common libraries here in play
"""
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.distance import pdist, squareform
import folium
from folium import plugins


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\tools\Anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\tools\Anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\tools\Anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start()
  File "c:\tools\Anaconda3\Lib\site-packages\tornado\platform\as

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.




A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\tools\Anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\tools\Anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\tools\Anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start()
  File "c:\tools\Anaconda3\Lib\site-packages\tornado\platform\as

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



ImportError: numpy.core.multiarray failed to import

In [95]:
data = pd.read_csv('./data/MTA_Subway_Stations_20241127.csv')

data.head()

,GTFS Stop ID,Station ID,Complex ID,Division,Line,Stop Name,Borough,CBD,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label,ADA,ADA Northbound,ADA Southbound,ADA Notes,Georeference
0,R01,1,1,BMT,Astoria,Astoria-Ditmars Blvd,Q,False,N W,Elevated,40.775036,-73.912034,Last Stop,Manhattan,0,0,0,NaN,POINT (-73.912034 40.775036)
1,R03,2,2,BMT,Astoria,Astoria Blvd,Q,False,N W,Elevated,40.770258,-73.917843,Astoria,Manhattan,1,1,1,NaN,POINT (-73.917843 40.770258)
2,R04,3,3,BMT,Astoria,30 Av,Q,False,N W,Elevated,40.766779,-73.921479,Astoria,Manhattan,0,0,0,NaN,POINT (-73.921479 40.766779)
3,R05,4,4,BMT,Astoria,Broadway,Q,False,N W,Elevated,40.761820,-73.925508,Astoria,Manhattan,0,0,0,NaN,POINT (-73.925508 40.76182)
4,R06,5,5,BMT,Astoria,36 Av,Q,False,N W,Elevated,40.756804,-73.929575,Astoria,Manhattan,0,0,0,NaN,POINT (-73.929575 40.756804)


We are going to clean up the data a little bit and drop useless features

In [96]:
data = data.drop(columns=['CBD', 
                          'Structure', 
                          'ADA', 
                          'ADA Northbound', 
                          'ADA Southbound', 
                          'ADA Notes', 
                          'Georeference', 
                          'GTFS Stop ID',
                          'Station ID',
                          'Division',
                          'Line'])

In [97]:
data.head()

,Complex ID,Stop Name,Borough,Daytime Routes,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label
0,1,Astoria-Ditmars Blvd,Q,N W,40.775036,-73.912034,Last Stop,Manhattan
1,2,Astoria Blvd,Q,N W,40.770258,-73.917843,Astoria,Manhattan
2,3,30 Av,Q,N W,40.766779,-73.921479,Astoria,Manhattan
3,4,Broadway,Q,N W,40.761820,-73.925508,Astoria,Manhattan
4,5,36 Av,Q,N W,40.756804,-73.929575,Astoria,Manhattan


Me and my homies hate SI lol

In [98]:
data = data[data['Borough'] != 'SI']
last_stops = data[data['North Direction Label'] == 'Last Stop']

last_stops.head()

,Complex ID,Stop Name,Borough,Daytime Routes,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label
0,1,Astoria-Ditmars Blvd,Q,N W,40.775036,-73.912034,Last Stop,Manhattan
114,618,8 Av,M,L,40.739777,-74.002578,Last Stop,Brooklyn
138,627,Franklin Av,Bk,S,40.680596,-73.955827,Last Stop,Prospect Park
141,143,Inwood-207 St,M,A,40.868072,-73.919899,Last Stop,Downtown
210,210,Norwood-205 St,Bx,D,40.874811,-73.878855,Last Stop,Manhattan


In [99]:
f_train = data[data['Daytime Routes'].str.contains('F')]

f_train

,Complex ID,Stop Name,Borough,Daytime Routes,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label
56,57,W 8 St-NY Aquarium,Bk,F Q,40.576127,-73.975939,Manhattan,Coney Island
57,58,Coney Island-Stillwell Av,Bk,D F N Q,40.577422,-73.981233,Manhattan,Last Stop
167,167,W 4 St-Wash Sq,M,B D F M,40.732338,-74.000495,Uptown,Downtown
174,636,Jay St-MetroTech,Bk,A C F,40.692338,-73.987342,Manhattan,Outbound
221,221,21 St-Queensbridge,Q,F,40.754203,-73.942836,Outbound,Manhattan
222,222,Roosevelt Island,M,F,40.759145,-73.953260,Queens,Downtown
223,223,Lexington Av/63 St,M,F Q,40.764629,-73.966113,Uptown & Queens,Downtown
224,224,57 St,M,F,40.763972,-73.977450,Uptown,Downtown
225,225,47-50 Sts-Rockefeller Ctr,M,B D F M,40.758663,-73.981329,Uptown,Downtown
226,609,42 St-Bryant Pk,M,B D F M,40.754222,-73.984569,Uptown,Downtown


Lets find a way to re order them to how the actual train line is.

In [100]:
def nearest_neighbor_path(distance_matrix, start=0):
    n = len(distance_matrix)
    path = [start]
    unvisited = set(range(n))
    unvisited.remove(start)
    
    while unvisited:
        current = path[-1]
        # Find the nearest unvisited station
        next_stop = min(unvisited, key=lambda x: distance_matrix[current][x])
        path.append(next_stop)
        unvisited.remove(next_stop)
    
    return path

def custom_sort(row):
    # Check both North and South labels
    if row['North Direction Label'] == 'Last Stop' or row['South Direction Label'] == 'Last Stop':
        return 'A'  # This will make 'Last Stop' appear first
    else:
        return row['North Direction Label']

f_train['sort_key'] = f_train.apply(custom_sort, axis=1)
f_train = f_train.sort_values('sort_key')
f_train = f_train.drop('sort_key', axis=1)

coords = f_train[['GTFS Latitude', 'GTFS Longitude']].values
distances = pdist(coords)
distance_matrix = squareform(distances)

f_train.head()

#print(coords)
#print(distances)
#print(distance_matrix)

C:\Users\vlcar\AppData\Local\Temp\ipykernel_25092\2138665790.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  f_train['sort_key'] = f_train.apply(custom_sort, axis=1)


,Complex ID,Stop Name,Borough,Daytime Routes,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label
57,58,Coney Island-Stillwell Av,Bk,D F N Q,40.577422,-73.981233,Manhattan,Last Stop
254,254,Jamaica-179 St,Q,F,40.712646,-73.783817,Last Stop,Manhattan
260,260,75 Av,Q,E F,40.718331,-73.837324,Jamaica,Manhattan
259,259,Kew Gardens-Union Tpke,Q,E F,40.714441,-73.831008,Jamaica,Manhattan
258,258,Briarwood,Q,E F,40.709179,-73.820574,Jamaica,Manhattan


In [101]:
path = nearest_neighbor_path(distance_matrix)

temp = f_train.iloc[path].reset_index(drop=True)

print("\nStations in order:")
for idx, row in temp.iterrows():
    print(f"{idx}. {row['Stop Name']} ({row['GTFS Latitude']}, {row['GTFS Longitude']})")


Stations in order:
0. Coney Island-Stillwell Av (40.577422, -73.981233)
1. W 8 St-NY Aquarium (40.576127, -73.975939)
2. Neptune Av (40.581011, -73.974574)
3. Avenue X (40.58962, -73.97425)
4. Avenue U (40.596063, -73.973357)
5. Kings Hwy (40.603217, -73.972361)
6. Avenue P (40.608944, -73.973022)
7. Avenue N (40.61514, -73.974197)
8. Bay Pkwy (40.620769, -73.975264)
9. Avenue I (40.625322, -73.976127)
10. 18 Av (40.629755, -73.976971)
11. Ditmas Av (40.636119, -73.978172)
12. Church Av (40.644041, -73.979678)
13. Fort Hamilton Pkwy (40.650782, -73.975776)
14. 15 St-Prospect Park (40.660365, -73.979493)
15. 7 Av (40.666271, -73.980305)
16. 4 Av-9 St (40.670272, -73.989779)
17. Smith-9 Sts (40.67358, -73.995959)
18. Carroll St (40.680303, -73.995048)
19. Bergen St (40.686145, -73.990862)
20. Jay St-MetroTech (40.692338, -73.987342)
21. York St (40.701397, -73.986751)
22. East Broadway (40.713715, -73.990173)
23. Delancey St-Essex St (40.718611, -73.988114)
24. 2 Av (40.723402, -73.9899

Now that we have the data cleaned and ordered for the F train, lets try to do it for every line.

In [102]:
all_trains = set()
for routes in data['Daytime Routes'].str.split():
    all_trains.update(routes)

all_trains = sorted(all_trains)
print('All trains', all_trains)

trains = {}
for route in all_trains:
    mask = data['Daytime Routes'].str.contains(route)
    temp = data[mask].copy()

    temp['sort_key'] = temp.apply(custom_sort, axis=1)
    temp = temp.sort_values('sort_key')
    temp = temp.drop('sort_key', axis=1)

    coords = temp[['GTFS Latitude', 'GTFS Longitude']].values
    distances = pdist(coords)
    distance_matrix = squareform(distances)

    path = nearest_neighbor_path(distance_matrix)

    final = temp.iloc[path].reset_index(drop=True)
    trains[route] = final

for route, df in trains.items():
    print(f"\nRoute {route} has {len(df)} stations:")
    print("Stations in order:")
    for idx, name in enumerate(df['Stop Name']):
        print(f"{idx+1}. {name}")

All trains ['1', '2', '3', '4', '5', '6', '7', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'J', 'L', 'M', 'N', 'Q', 'R', 'S', 'W', 'Z']

Route 1 has 38 stations:
Stations in order:
1. Van Cortlandt Park-242 St
2. 238 St
3. 231 St
4. Marble Hill-225 St
5. 215 St
6. 207 St
7. Dyckman St
8. 191 St
9. 181 St
10. 168 St-Washington Hts
11. 157 St
12. 145 St
13. 137 St-City College
14. 125 St
15. 116 St-Columbia University
16. Cathedral Pkwy (110 St)
17. 103 St
18. 96 St
19. 86 St
20. 79 St
21. 72 St
22. 66 St-Lincoln Center
23. 59 St-Columbus Circle
24. 50 St
25. Times Sq-42 St
26. 34 St-Penn Station
27. 28 St
28. 23 St
29. 18 St
30. 14 St
31. Christopher St-Stonewall
32. Houston St
33. Canal St
34. Franklin St
35. Chambers St
36. WTC Cortlandt
37. Rector St
38. South Ferry

Route 2 has 49 stations:
Stations in order:
1. Flatbush Av-Brooklyn College
2. Newkirk Av-Little Haiti
3. Beverly Rd
4. Church Av
5. Winthrop St
6. Sterling St
7. President St-Medgar Evers College
8. Franklin Av-Medgar Evers Coll

In [103]:
def create_subway_graph(trains_dict):
    """
    Create a subway graph using Complex ID as the unique identifier for stations.
    """
    G = nx.Graph()
    
    # Add nodes and edges for each train route
    for route, df in trains_dict.items():
        # Add all stations as nodes with their attributes
        for idx, row in df.iterrows():
            if not G.has_node(row['Complex ID']):
                G.add_node(row['Complex ID'],
                          name=row['Stop Name'],
                          borough=row['Borough'],
                          latitude=row['GTFS Latitude'],
                          longitude=row['GTFS Longitude'],
                          routes={route})
            else:
                # If node exists, add this route to its routes set
                G.nodes[row['Complex ID']]['routes'].add(route)
        
        # Connect consecutive stations in the route
        for i in range(len(df) - 1):
            G.add_edge(df['Complex ID'].iloc[i],
                      df['Complex ID'].iloc[i + 1],
                      route=route)
    
    return G

# Create the graph
G = create_subway_graph(trains)

# Print basic information
print(f"Number of stations (nodes): {G.number_of_nodes()}")
print(f"Number of connections (edges): {G.number_of_edges()}")

# Print some example nodes
print("\nExample stations:")
for complex_id in list(G.nodes)[:5]:  # First 5 stations
    node_data = G.nodes[complex_id]
    print(f"\nComplex ID: {complex_id}")
    print(f"Station Name: {node_data['name']}")
    print(f"Routes: {sorted(node_data['routes'])}")
    print(f"Borough: {node_data['borough']}")

# Print transfer stations (stations with multiple routes)
print("\nTransfer stations:")
for node in G.nodes:
    routes = G.nodes[node]['routes']
    if len(routes) > 1:
        print(f"{G.nodes[node]['name']} (Complex ID: {node}): {sorted(routes)}")

Number of stations (nodes): 424
Number of connections (edges): 522

Example stations:

Complex ID: 293
Station Name: Van Cortlandt Park-242 St
Routes: ['1']
Borough: Bx

Complex ID: 294
Station Name: 238 St
Routes: ['1']
Borough: Bx

Complex ID: 295
Station Name: 231 St
Routes: ['1']
Borough: Bx

Complex ID: 296
Station Name: Marble Hill-225 St
Routes: ['1']
Borough: M

Complex ID: 297
Station Name: 215 St
Routes: ['1']
Borough: M

Transfer stations:
168 St-Washington Hts (Complex ID: 605): ['1', 'A', 'C']
96 St (Complex ID: 310): ['1', '2', '3']
72 St (Complex ID: 313): ['1', '2', '3']
59 St-Columbus Circle (Complex ID: 614): ['1', 'A', 'B', 'C', 'D']
Times Sq-42 St (Complex ID: 611): ['1', '2', '3', '7', 'A', 'C', 'E', 'N', 'Q', 'R', 'S', 'W']
34 St-Penn Station (Complex ID: 318): ['1', '2', '3']
14 St (Complex ID: 601): ['1', '2', '3', 'F', 'L', 'M']
Chambers St (Complex ID: 327): ['1', '2', '3']
South Ferry (Complex ID: 635): ['1', 'R', 'W']
Flatbush Av-Brooklyn College (Complex ID

Calculate network centrality metrics to identify key stations:

In [104]:
def analyze_station_centrality(G):
    """
    Calculate various centrality metrics for stations.
    """
    metrics = {
        'degree': nx.degree_centrality(G),
        'betweenness': nx.betweenness_centrality(G),
        'closeness': nx.closeness_centrality(G)
    }
    
    # Combine metrics into a DataFrame
    df_metrics = pd.DataFrame(metrics)
    df_metrics['station_name'] = [G.nodes[node]['name'] for node in G.nodes()]
    df_metrics['routes'] = [sorted(G.nodes[node]['routes']) for node in G.nodes()]
    
    return df_metrics.sort_values('betweenness', ascending=False)

# Calculate and display top stations by centrality
centrality_metrics = analyze_station_centrality(G)
print("Top 10 most central stations:")
print(centrality_metrics.head(10))

Top 10 most central stations:
       degree  betweenness  closeness                      station_name  \
610  0.018913     0.431494   0.143002               Grand Central-42 St   
613  0.016548     0.353317   0.135882                             59 St   
611  0.030733     0.300853   0.136232                    Times Sq-42 St   
42   0.011820     0.226528   0.135751                     Prospect Park   
392  0.009456     0.213999   0.117045                            125 St   
602  0.023641     0.212027   0.137965                    14 St-Union Sq   
397  0.009456     0.209435   0.125074                             86 St   
617  0.021277     0.205707   0.126231          Atlantic Av-Barclays Ctr   
626  0.016548     0.200081   0.131530  Franklin Av-Medgar Evers College   
603  0.009456     0.164597   0.102995            149 St-Grand Concourse   

                                   routes  
610                       [4, 5, 6, 7, S]  
613                    [4, 5, 6, N, R, W]  
611  [1, 2, 

Identify important transfer corridors:

In [105]:
def analyze_transfer_corridors(G):
    """
    Find and analyze important transfer corridors.
    """
    transfer_stations = []
    
    for node in G.nodes():
        routes = G.nodes[node]['routes']
        if len(routes) > 1:
            neighbors = list(G.neighbors(node))
            neighbor_routes = [G.nodes[n]['routes'] for n in neighbors]
            
            transfer_stations.append({
                'station_id': node,
                'name': G.nodes[node]['name'],
                'routes': sorted(routes),
                'num_routes': len(routes),
                'num_neighbors': len(neighbors),
                'connected_routes': sorted(set.union(*neighbor_routes))
            })
    
    return pd.DataFrame(transfer_stations).sort_values('num_routes', ascending=False)